0.0 Imports

In [28]:
import pandas as pd
import numpy as np

0.1 functions

0.2 Loading Dataset

In [29]:
data = pd.read_csv('../dataset/train.csv')
data.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0


1.0 data descriptionn

In [30]:
df1 = data.copy()
df1 = df1.rename(columns=lambda x: x.lower())            


1.1 Data Dimension

In [31]:
print('Number of Rows: {}'.format(df1.shape[0]))
print('Number of Col: {}'.format(df1.shape[1]))

Number of Rows: 381109
Number of Col: 12


1.2 data types

In [32]:
df1.dtypes

id                        int64
gender                   object
age                       int64
driving_license           int64
region_code             float64
previously_insured        int64
vehicle_age              object
vehicle_damage           object
annual_premium          float64
policy_sales_channel    float64
vintage                   int64
response                  int64
dtype: object

1.3 Check NA

In [33]:
df1.isna().sum() #/ len(df1)*100

id                      0
gender                  0
age                     0
driving_license         0
region_code             0
previously_insured      0
vehicle_age             0
vehicle_damage          0
annual_premium          0
policy_sales_channel    0
vintage                 0
response                0
dtype: int64

1.4 Data Descriptive

In [34]:
df1.describe()

,id,age,driving_license,region_code,previously_insured,annual_premium,policy_sales_channel,vintage,response
count,381109.000000,381109.000000,381109.000000,381109.000000,381109.000000,381109.000000,381109.000000,381109.000000,381109.000000
mean,190555.000000,38.822584,0.997869,26.388807,0.458210,30564.389581,112.034295,154.347397,0.122563
std,110016.836208,15.511611,0.046110,13.229888,0.498251,17213.155057,54.203995,83.671304,0.327936
min,1.000000,20.000000,0.000000,0.000000,0.000000,2630.000000,1.000000,10.000000,0.000000
25%,95278.000000,25.000000,1.000000,15.000000,0.000000,24405.000000,29.000000,82.000000,0.000000
50%,190555.000000,36.000000,1.000000,28.000000,0.000000,31669.000000,133.000000,154.000000,0.000000
75%,285832.000000,49.000000,1.000000,35.000000,1.000000,39400.000000,152.000000,227.000000,0.000000
max,381109.000000,85.000000,1.000000,52.000000,1.000000,540165.000000,163.000000,299.000000,1.000000


In [35]:
num_attributes = df1.select_dtypes( include=['int64','float64'] )
cat_attributes = df1.select_dtypes( exclude= ['int64', 'float64'] )

In [36]:
# Central tendency - mean, mediana
ct1 = pd.DataFrame(num_attributes.apply(np.mean))
ct2 = pd.DataFrame(num_attributes.apply(np.median))

# dispersion - std,num_attributesge, skew, kurtosis
d1  = pd.DataFrame(num_attributes.apply(np.std))
d2  = pd.DataFrame(num_attributes.apply(min))
d3  = pd.DataFrame(num_attributes.apply(max))
d4  = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min()))
d5  = pd.DataFrame(num_attributes.apply(lambda x: x.skew()))
d6  = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis()))

# concatenar
m = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6], axis=1)
# Renomear as colunas
m.columns = ['min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']

# Exibir o DataFrame com as colunas renomeadas
m

,min,max,range,mean,median,std,skew,kurtosis
id,1.0,381109.0,381108.0,190555.000000,190555.0,110016.691870,9.443274e-16,-1.200000
age,20.0,85.0,65.0,38.822584,36.0,15.511591,6.725390e-01,-0.565655
driving_license,0.0,1.0,1.0,0.997869,1.0,0.046109,-2.159518e+01,464.354302
region_code,0.0,52.0,52.0,26.388807,28.0,13.229871,-1.152664e-01,-0.867857
previously_insured,0.0,1.0,1.0,0.458210,0.0,0.498251,1.677471e-01,-1.971871
annual_premium,2630.0,540165.0,537535.0,30564.389581,31669.0,17213.132474,1.766087e+00,34.004569
policy_sales_channel,1.0,163.0,162.0,112.034295,133.0,54.203924,-9.000081e-01,-0.970810
vintage,10.0,299.0,289.0,154.347397,154.0,83.671194,3.029517e-03,-1.200688
response,0.0,1.0,1.0,0.122563,0.0,0.327935,2.301906e+00,3.298788


2.0 Feature Enginier

In [ ]:
df2 = df1.copy()

In [ ]:
df2['vehicle_age'].unique()

In [ ]:
# Convertendo a coluna para tipo string
df2['vehicle_damage'] = df2['vehicle_damage'].map({'Yes': 1, 'No': 0})

# Transformando em over e betwen
df2['vehicle_age'] = df2['vehicle_age'].apply(lambda x: 'over_2_years' if x == '> 2 Years' else 'between_1_2_year' 
                                                                       if x == '1-2 Year' else 'below_1_year')


# Removendo os sinais < e >
#df2['vehicle_age'] = df2['vehicle_age'].str.replace('[<>]', '', regex=True)


# Exibindo o DataFrame resultante
df2.head()

3.0 Data Filtering

4.0 EDA

5.0 Feature Selectio

6.0 Data Preparation


7.0 Machine Learning